#### 전이 학습
- 거대 이미지 데이터셋으로 사전 학습 된 모델을 활용
- ImageNet 데이터베이스 기반 모델들 사용
- 사용법
    * 모델의 특징추출부분(CNN)은 재사용
    * 모델의 학습 및 분류부분(FC)은 커스텀화 사용

- [1] 모듈 로딩 <hr>

In [12]:
## 모듈 로딩
import torch
import torch.nn as nn
import torchvision.models as models     # 사전학습된 모델들 관련
from torchinfo import summary           # 모델 정보 관련

- [2] 사전학습된 모델 로딩 및 설정 <hr>
    - 모델에 사용할 가중치 및 절편 설정
    - 층별 가중치 및 절편 비활성화 설정

In [4]:
## 사전학습된 모델 로딩
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\kdp/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100.0%


In [8]:
## 모델 구조 확인
print(model)
summary(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
## 사전학습된 모델의 파라미터 비활성화 설정
for named, param in model.named_parameters():
    print(f'[{named}] = {param.shape}')
    # 역전파 시에 업데이트 되지 않도록 설정
    param.requires_grad = False

[features.0.weight] = torch.Size([64, 3, 3, 3])
[features.0.bias] = torch.Size([64])
[features.2.weight] = torch.Size([64, 64, 3, 3])
[features.2.bias] = torch.Size([64])
[features.5.weight] = torch.Size([128, 64, 3, 3])
[features.5.bias] = torch.Size([128])
[features.7.weight] = torch.Size([128, 128, 3, 3])
[features.7.bias] = torch.Size([128])
[features.10.weight] = torch.Size([256, 128, 3, 3])
[features.10.bias] = torch.Size([256])
[features.12.weight] = torch.Size([256, 256, 3, 3])
[features.12.bias] = torch.Size([256])
[features.14.weight] = torch.Size([256, 256, 3, 3])
[features.14.bias] = torch.Size([256])
[features.17.weight] = torch.Size([512, 256, 3, 3])
[features.17.bias] = torch.Size([512])
[features.19.weight] = torch.Size([512, 512, 3, 3])
[features.19.bias] = torch.Size([512])
[features.21.weight] = torch.Size([512, 512, 3, 3])
[features.21.bias] = torch.Size([512])
[features.24.weight] = torch.Size([512, 512, 3, 3])
[features.24.bias] = torch.Size([512])
[features.26.we

- [3] 사전학습모델의 분류 관련층을 커스텀화 진행 <hr>

In [11]:
model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [19]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [18]:
class My(nn.Module):
    def __init__(self):
        super().__init__()
        self.feautures = nn.Sequential(
            nn.Conv2d(),
            nn.Conv2d(),
            nn.ReLU(),
            nn.Conv2d(),
            nn.Conv2d(),
            nn.ReLU()
        )
        self.classifier =nn.Sequential(
            nn.Linear(3,5),
            nn.ReLU(),
            nn.Linear(3,5),
            nn.ReLU()
        )
    def forward(self, data):
        output = self.feautures(data)
        output = output.view(output.shape[0],-1)
        return self.classifier(output)